### PROBLEM STATEMENT




Many UFO's are visible on a video frame with different numbers ranging from 0 - 9 written on them. The task is to find the location of those UFO's on each frame, sort them from top to bottom by their position and return numbers written on them. 



![title](frame.jpg)

The goal of this detection technique is to determine where objects are located in a given image called as object localisation and which category each object belongs to, that is called as object classification.   

In [1]:
pip -V

pip 21.1.1 from /Applications/anaconda3/lib/python3.7/site-packages/pip (python 3.7)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


### 1. Read the file using OpenCV and create an instance.

In [1]:
import cv2

img_file = 'frame.jpg'

# create an opencv image
img = cv2.imread(img_file)

# convert the image to gray scale format

img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# applying canny edge detection
edged = cv2.Canny(img, 10, 250)

#finding contours
(cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

def get_contour_precedence(contour, cols):
    tolerance_factor = 10
    origin = cv2.boundingRect(contour)
    return ((origin[1] // tolerance_factor) * tolerance_factor) * cols + origin[0]

cnts.sort(key=lambda x:get_contour_precedence(x, img.shape[1]))

# For debugging purposes.
#for i in range(len(cnts)):
    #img = cv2.putText(img, str(i), cv2.boundingRect(cnts[i])[:2], cv2.FONT_HERSHEY_COMPLEX, 1, [125])
    

idx = 0
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    if w>50 and h>50:
        idx+=1
        new_img=img[y:y+h,x:x+w]
        #cropping images
        cv2.imwrite("cropped/"+str(idx) + '.png', new_img)
#cv2.imshow("Original Image",image)
#cv2.imshow("Canny Edge",edged)
#cv2.waitKey(0)
print('>> Objects Cropped Successfully!')
print(">> Check out 'cropped' Directory")

In [2]:
img.shape

(506, 900, 3)

In [ ]:
'''
from above we can infer that :-

A. img is the digital representation of image interms of pixel intensities ranging from 0-255
B. The resolution of the image. is 506X900 , where each pixel value is of depth 3 containing values for 3 primary
colours Red,Blue,Green (RBG).
'''

### 2. Define the classifiers 

In [ ]:
'''

APPROACH 1 -

HAAR CASCADE CLASSIFIERS are an effective way for object detection.Haar Cascade is a machine learning-based 
approach where a lot of positive and negative images are used to train the classifier.

1. Positive images – These images contain the images which we want our classifier to identify.
2. Negative Images – Images of everything else, which do not contain the object we want to detect.

In total we will be require 10 HAAR cascade classifiers that will identify the features of the objects and therefore
the object types ranging from 0-9.

Below are our custom made HAAR CASCADE CLASSIFIERS.

'''

In [ ]:
'''
APPROACH 2 FOR OBJECT DETECTION -

Knowing where an object is in an image is called localization in computer vision. 
Using contour detection, we can detect the borders of objects, and therefore, localize them easily.

we are going to do contours and contour detection using OpenCV(coding in both Python).

1.OpenCV makes it really easy for finding, and drawing contours in images. It provides two simple 
functions to do so: findContours(), and drawContours().

'''

In [3]:
# convert the image to gray scale format

img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


In [4]:
img_gray

array([[143, 142, 144, ..., 214, 214, 215],
       [140, 140, 142, ..., 213, 213, 214],
       [139, 139, 141, ..., 213, 213, 214],
       ...,
       [105, 102, 105, ..., 115, 116, 118],
       [104, 102, 105, ..., 116, 116, 118],
       [109, 106, 109, ..., 118, 119, 121]], dtype=uint8)

In [ ]:
'''
we are using OpenCV’s cvtColor() function to convert the original RGB image to a grayscale image. this will help 
to apply binary thresholding and in obtaining much better results.
'''

In [ ]:
# apply binary thresholding

ret, thresh = cv2.threshold(img_gray, 150, 255, cv2.THRESH_BINARY)

# visualize the binary image

cv2.imshow('Binary image', thresh)

cv2.waitKey(0)

cv2.imwrite('image_thres1.jpg', thresh)

cv2.destroyAllWindows()


In [ ]:
'''
We are using the threshold() function to apply binary thresholding to the image. 
Any pixel with a value greater than 150 will be updated  to a value of 255, that is, 
completely white. And, all the other pixels in the resulting image will be 0, that is, black. 

'''

In [ ]:
#let us find and draw the contours using the CHAIN_APPROX_NONE method. 

contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)

# draw contours on the original image

image_copy = image.copy()

cv2.drawContours(image=image_copy, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

# see the results

cv2.imshow('None approximation', image_copy)

cv2.waitKey(0)

cv2.imwrite('contours_none_image1.jpg', image_copy)

cv2.destroyAllWindows()


In [ ]:
# to actually visualize the effect of `CHAIN_APPROX_SIMPLE`, we need a proper image

image1 = cv2.imread('frame.jpg')

img_gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)

ret, thresh1 = cv2.threshold(img_gray1, 150, 255, cv2.THRESH_BINARY)

contours2, hierarchy2 = cv2.findContours(thresh1, cv2.RETR_TREE,

                                               cv2.CHAIN_APPROX_SIMPLE)

image_copy2 = image1.copy()

cv2.drawContours(image_copy2, contours2, -1, (0, 255, 0), 2, cv2.LINE_AA)

cv2.imshow('SIMPLE Approximation contours', image_copy2)

cv2.waitKey(0)

image_copy3 = image1.copy()

for i, contour in enumerate(contours2): # loop over one contour area
    for j, contour_point in enumerate(contour): # loop over the points

       # draw a circle on the current contour coordinate

       cv2.circle(image_copy3, ((contour_point[0][0], contour_point[0][1])), 2, (0, 255, 0), 2, cv2.LINE_AA)

# see the results

cv2.imshow('CHAIN_APPROX_SIMPLE Point only', image_copy3)

cv2.waitKey(0)

cv2.imwrite('contour_point_simple.jpg', image_copy3)

cv2.destroyAllWindows()

In [1]:
1/2

0.5

In [5]:
# applying canny edge detection
edged = cv2.Canny(img, 10, 250)

#finding contours
(cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


In [25]:
img

array([[[ 77, 141, 172],
        [ 76, 140, 171],
        [ 77, 141, 175],
        ...,
        [180, 213, 229],
        [180, 213, 229],
        [181, 214, 230]],

       [[ 74, 138, 169],
        [ 74, 138, 169],
        [ 75, 139, 173],
        ...,
        [179, 212, 228],
        [179, 212, 228],
        [180, 213, 229]],

       [[ 73, 137, 168],
        [ 73, 137, 168],
        [ 74, 138, 172],
        ...,
        [179, 212, 228],
        [179, 212, 228],
        [180, 213, 229]],

       ...,

       [[ 41, 115, 109],
        [ 38, 112, 106],
        [ 41, 115, 109],
        ...,
        [ 49, 125, 121],
        [ 50, 126, 122],
        [ 52, 128, 124]],

       [[ 40, 114, 108],
        [ 38, 112, 106],
        [ 41, 115, 109],
        ...,
        [ 50, 126, 122],
        [ 50, 126, 122],
        [ 52, 128, 124]],

       [[ 45, 119, 113],
        [ 42, 116, 110],
        [ 45, 119, 113],
        ...,
        [ 52, 128, 124],
        [ 53, 129, 125],
        [ 55, 131, 127]]

In [6]:
def get_contour_precedence(contour, cols):
    tolerance_factor = 10
    origin = cv2.boundingRect(contour)
    return ((origin[1] // tolerance_factor) * tolerance_factor) * cols + origin[0]

cnts.sort(key=lambda x:get_contour_precedence(x, img.shape[1]))

# For debugging purposes.
#for i in range(len(cnts)):
    #img = cv2.putText(img, str(i), cv2.boundingRect(cnts[i])[:2], cv2.FONT_HERSHEY_COMPLEX, 1, [125])

In [29]:
img

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[  0, 255, 255],
        [  0, 255, 255],
        [  0, 255, 255],
        ...,
        [  0, 255, 255],
        [  0, 255, 255],
        [  0, 255, 255]],

       [[  0, 255, 255],
        [  0, 255, 255],
        [  0, 255, 255],
        ...,
        [  0, 255, 255],
        [  0, 255, 255],
        [  0, 255, 255]],

       [[  0, 255, 255],
        [  0, 255, 255],
        [  0, 255, 255],
        ...,
        [  0, 255, 255],
        [  0, 255, 255],
        [  0, 255, 255]]

In [37]:
cv2.imwrite("cropped/"+'contours_sort.png', img)

True

In [7]:
idx = 0
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    if w>50 and h>50:
        idx+=1
        new_img=img[y:y+h,x:x+w]
        #cropping images
        cv2.imwrite("cropped/"+str(idx) + '.png', new_img)
#cv2.imshow("Original Image",image)
#cv2.imshow("Canny Edge",edged)
#cv2.waitKey(0)
print('>> Objects Cropped Successfully!')
print(">> Check out 'cropped' Directory")

>> Objects Cropped Successfully!
>> Check out 'cropped' Directory


In [9]:
img_1 = cv2.imread('2.png', 0)
img_2 = cv2.imread('img1.jpg', 0)

ret, thresh = cv2.threshold(img_1, 127, 255, 0)
ret, thresh2 = cv2.threshold(img_2, 127, 255, 0)
contours1, hierarchy = cv2.findContours(thresh, 2, 1)
print(len(contours1))
cnt1 = contours1[1]
contours2, hierarchy = cv2.findContours(thresh2, 2, 1)
print(len(contours2))
cnt2 = contours2[1]
ret = cv2.matchShapes(cnt1, cnt2, 1, 0.0)

print(ret)


37
10
0.0


In [ ]:
original_image = cv2.imread('frame.jpg')
original_copy = original_image.copy()
image = cv2.imread('frame.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
canny = cv2.Canny(gray, 130, 255, 1)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
dilate = cv2.dilate(canny, kernel, iterations=1)

cnts = cv2.findContours(dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.01 * peri, True)
    x,y,w,h = cv2.boundingRect(approx)
    aspect_ratio = w / float(h)

    if (aspect_ratio >= 0.8 and aspect_ratio <= 1.6):
        ROI = original_copy[y:y+h, x:x+w]
        cv2.rectangle(original_image,(x,y),(x+w,y+h),(0,255,0),3)

cv2.imshow('canny', canny)
cv2.imshow('dilate', dilate)
cv2.imshow('original image', original_image)
cv2.imshow('ROI', ROI)
cv2.waitKey(0)